https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0

In [83]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from nltk.corpus import brown
import numpy as np
from sklearn.model_selection import train_test_split

In [84]:
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')
print(len(pd.Series(brown_news_tagged).unique()))
len(brown.words())

15114


1161192

In [85]:
def tensor_to_array(t):
    return np.array([i.item() for i in list(t)])

In [86]:
# Prepare Corpus
DATA_LIMIT = None

remove_punc = [word for word in brown_news_tagged]
text = pd.Series([word[0].lower() for word in remove_punc][:DATA_LIMIT])
len(text)

100554

In [87]:
#filter out rare words
N_times = 25
v_counts = text.value_counts()
corpus = pd.Series([key for key in v_counts.keys() if v_counts[key] >= N_times])
len(corpus)


432

In [125]:
vocab = {}
for i in range(len(corpus)):
    vocab[corpus[i]] = i+1
len(vocab)

432

In [97]:
vocab["<unk>"] = 0

In [98]:
enc_text = [vocab.setdefault(word, 0) for word in text]

In [99]:
SCANNING_WINDOW = 5
#maybe need to split into paragraphs b/c different topics...
#returns context, middle word
def get_data(index, window, data):
    return list(data[index-window:index])+list(data[index+1:index+window+1]), data[index]

In [113]:
x = []
y = []
for i in range(SCANNING_WINDOW, len(enc_text) - SCANNING_WINDOW):
    a, b = get_data(i, SCANNING_WINDOW, enc_text)
    x.append(a)
    y.append(b)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=5)

In [156]:
EMBED_DIMENSION = 300
EMBED_MAX_NORM = 1
class Net_CBOW(nn.Module):
    def __init__(self, vocab_size: int):
        super(Net_CBOW, self).__init__()
        self.embeddings = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=EMBED_DIMENSION,
            max_norm=EMBED_MAX_NORM,
        )
        self.linear = nn.Linear(
            in_features=EMBED_DIMENSION,
            out_features=vocab_size,
        )
    def forward(self, inputs_):
        x = self.embeddings(inputs_)
        x = x.mean(axis=0)
        x = self.linear(x)
        return x

In [154]:
net = Net_CBOW(len(vocab))
params = list(net.parameters())
net.zero_grad()
criterion = nn.CrossEntropyLoss()
losses = []
optimizer = optim.Adam(net.parameters(), lr=0.025)
NUM_EPOCHS = 5
scheduler = optim.lr_scheduler.LinearLR(optimizer, 1.0, 0.0, total_iters=NUM_EPOCHS, verbose=True)


In [155]:

for epoch in range(NUM_EPOCHS):
    print("RUN", str(epoch+1)+"/"+str(NUM_EPOCHS), end=": ")
    for index in range(len(x_train)):
        if index % (len(x_train)//10) == 0:
            print("•", end="")

        context, target = x_train[index], y_train[index]
        optimizer.zero_grad()   # zero the gradient buffers
        output = net(torch.tensor(context))
        loss = criterion(output, torch.tensor(target))
        loss.backward()
        optimizer.step()    # Does the update

    for context, target in zip(x_test, y_test):
        output = net(torch.tensor(context))
        losses.append(criterion(output, torch.tensor(target)).item())

    scheduler.step()
    print()

RUN 1/5: •

IndexError: Target 432 is out of bounds.